In [4]:
import csv 
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from selenium import webdriver
import re
from selenium.webdriver.common.by import By


In [5]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element(By.XPATH,'.//span').text
    try:
        handle = card.find_element(By.XPATH,'.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element(By.XPATH,'.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    # comment = card.find_element(By.XPATH,'.//div[2]/div[2]/div[1]').text
    # # print(comment)
    # responding = card.find_element(By.XPATH,'.//div[2]/div[2]/div[2]').text
    # text = comment + responding
    tweetText = card.find_element(By.XPATH, ".//div[@data-testid='tweetText']").text

    reply_cnt = card.find_element(By.XPATH,'.//button[@data-testid="reply"]').text
    retweet_cnt = card.find_element(By.XPATH,'.//button[@data-testid="retweet"]').text
    like_cnt = card.find_element(By.XPATH,'.//button[@data-testid="like"]').text
    # print(text)
    # get a string of all emojis contained in the tweet
    """Emojis are stored as images... so I convert the filename, which is stored as unicode, into 
    the emoji character."""
    emoji_tags = card.find_elements(By.XPATH,'.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = (username, handle, postdate, tweetText, emojis, reply_cnt, retweet_cnt, like_cnt)
    return tweet    

In [12]:
from time import sleep

# Initialize the Chrome driver
driver = webdriver.Chrome()

# Open Twitter login page
driver.get('https://www.twitter.com/login')
sleep(5)

driver.maximize_window()
sleep(5)

# Find and input username
username = driver.find_element("xpath", '//input[@name="text"]')
username.send_keys('@xxx')
sleep(3)
username.send_keys(Keys.RETURN)
sleep(5)

# Find and input password
password = driver.find_element("xpath", '//input[@name="password"]')
password.send_keys('xxxxxxx!')
password.send_keys(Keys.RETURN)
sleep(3)

# Find the search input and enter 'indosat'
search_input = driver.find_element("xpath", '//input[@aria-label="Search query"]')
search_input.send_keys('Topic')
sleep(1)
search_input.send_keys(Keys.RETURN)
sleep(3)

# Click on the 'Latest' tab
driver.find_element("link text", 'Latest').click()
sleep(3)

# Initialize data and tweet IDs
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

# Define the scroll interval (adjust as needed)
scroll_interval = 5  # seconds

while scrolling:
    page_cards = driver.find_elements(By.XPATH, ".//article[@data-testid='tweet']")
    for card in page_cards[-20:]:
        tweet = get_tweet_data(card)
        if tweet is None:
            continue
        else:
            if tweet[1] == tweet[2]:
                continue
            else:
                if "filter account" in tweet[1].lower() or "filter account" in tweet[1].lower():
                    continue
                else:
                    if "filter username" in tweet[0].lower() or "filter username" in tweet[0].lower():
                        # Code to execute if "telkomsel" (case-insensitive) is found in tweet[0]
                        continue
                    else:
                        if tweet[3].lower()=="filter tweet" or tweet[3].lower()=="filter tweet" or tweet[3].lower()=='filter tweet': #avoid tweet which only contains tagging to official account
                            continue
                        else:                    
                            if "filter tweet" in tweet[3].lower() or "filter tweet" in tweet[3].lower() or "filter tweet" in tweet[3].lower() or "filter tweet" in tweet[3].lower():
                                if tweet:
                                    tweet_id = ''.join(tweet)
                                    if tweet_id not in tweet_ids:
                                        tweet_ids.add(tweet_id)
                                        data.append(tweet)
                                        print(tweet)

    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(scroll_interval)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            if scroll_attempt >= 8:
                scrolling = False
                break
            else:
                sleep(scroll_interval)
        else:
            last_position = curr_position
            break


('Irv', '@iirrrvn', '2024-07-31T05:54:01.000Z', 'Dari pagi ngecek pilihan paket telkomsel, barusan buka mytelkomsel lg trs kaget ada yg Rp0, iseng coba beli ternyata berhasil \nCuma 1gb/bulan sih selama 180 hari, lumayan bgt buat nambahin masa aktif wkwk', '🍉 😭', '', '', '')
('lele semigap era', '@nelakilometer', '2024-07-31T05:12:35.000Z', 'sumpah ye my telkomsel knp cuma bisa gopay sekarang  masa gabisa bca si', '😭 😭 😭', '2', '', '')
('Aw Kayin', '@kayincsy', '2024-07-31T03:25:31.000Z', 'Hai kak, saya menggunakan aplikasi my Telkomsel. Untuk bagian pengecekan bagaimana ya?', '', '1', '', '')
('Indra', '@indraw151', '2024-07-30T19:14:29.000Z', 'Sudah saya minta bantuan di my Telkomsel gak di jawab kak, diam mereka dengan tiada jawaban. Tolong kak di proses..', '🏦 🏦', '1', '', '')
('', '@zhras_', '2024-07-30T18:41:11.000Z', 'last day check in my telkomsel, mari kita claim semua', '🌹 😂', '', '', '2')
('Indra', '@indraw151', '2024-07-30T18:07:11.000Z', '@My_Telkomsel\n saya isi pulsa mel

In [25]:
print(len(data))

424


To Continue Scrap after get limited by X

In [24]:
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True


scroll_interval = 5  # seconds

while scrolling:
    page_cards = driver.find_elements(By.XPATH, ".//article[@data-testid='tweet']")
    for card in page_cards[-20:]:
        tweet = get_tweet_data(card)
        if tweet is None:
            continue
        else:
            if tweet[1] == tweet[2]:
                continue
            else:
                if "filter 1" in tweet[1].lower() or "filter 1" in tweet[1].lower():
                    continue
                else:
                    if "filter 2" in tweet[0].lower() or "filter 2" in tweet[0].lower():
                        # Code to execute if "telkomsel" (case-insensitive) is found in tweet[0]
                        continue
                    else:
                        if tweet[3].lower()=="filter 3" or tweet[3].lower()=="filter 3" or tweet[3].lower()=='filter 3':
                            continue
                        else:                    
                            if "filter 4" in tweet[3].lower() or "filter 4" in tweet[3].lower() or "filter 4" in tweet[3].lower() or "filter 4" in tweet[3].lower():
                                if tweet:
                                    tweet_id = ''.join(tweet)
                                    if tweet_id not in tweet_ids:
                                        tweet_ids.add(tweet_id)
                                        data.append(tweet)
                                        print(tweet)

    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(scroll_interval)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            if scroll_attempt >= 8:
                scrolling = False
                break
            else:
                sleep(scroll_interval)
        else:
            last_position = curr_position
            break

('myonlysunshine', '@Lees46215', '2024-07-18T01:13:25.000Z', 'Paket telkomsel yg combo sakti 30an GB tu udah nggak ada lagi kah??barusan liat di my telkomsel udah nggak ada..beli yg paket OMG yg 75rb di keterangan ny bisa 4gb sampe 14gb eh pas diaktifkn beneran cuma 4gb donk..wkwkkwk,,sisanya yg OMG tuh nggak tau spy kepake ny gimana ', '😅', '1', '', '')
('Jena', '@hanjiesace', '2024-07-18T01:11:11.000Z', 'udah coba hubungi cs di apk my telkomsel tetep aja ga ngasih solusi kenapa kuotanya gabisa dipake. udah gitu kuota yang awalnya dalam sebulan selalu sisa tiba tiba dalam 2 minggu udah gabisa dipake, kan sayang banget duit gue.. terus juga tampilan aplikasinya yang baru aneh—', '🥠', '1', '', '')
('Monza Aulia', '@MonzaAulia', '2024-07-18T00:15:29.000Z', 'Beli paket internet telkomsel, pembayaran di \n@gopayindonesia\n sukses, tapi di status di app My \n@Telkomsel\n malah masih menunggu pembayaran. Ini kenapa ya?', '', '2', '', '')
('', '@babystarcandybb', '2024-07-18T00:12:57.000Z', '

In [26]:
# Get the current date and time
import datetime

current_datetime = datetime.datetime.now()

# Format the current date and time as a string (e.g., "2023-09-07_12-34")
current_date_time_str = current_datetime.strftime('%Y-%m-%d_%H-%M')

# Construct the file name with the current date and time
file_name = f'namefile_{current_date_time_str}.csv'

with open(file_name, 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Emojis', 'Comments', 'Likes', 'Retweets']
    
    # Specify the delimiter as a semicolon (;)
    writer = csv.writer(f, delimiter=';')
    
    writer.writerow(header)
    writer.writerows(data)